In [1]:
import os
import json

In [4]:
data = ['a b b\n','b\n','c\n']
#单层列表写入文件
with open("test_textdata.txt","w") as f:
    f.writelines(data)

In [2]:
def write_txt(data,save_dir):
    with open(save_dir,"w") as f:
        f.writelines(data)
        
def generate_label(bbox,img_w,img_h,class_id):
    x1,y1 = bbox[0]
    x2,y2 = bbox[1]
    
    bbox_w = x2-x1
    bbox_h = y2-y1
    x_center = (x1+x2)/2
    y_center = (y1+y2)/2
    
    x_center /= img_w
    y_center /= img_h
    bbox_w /= img_w
    bbox_h /= img_h
    
    label = "{} {} {} {} {}\n".format(class_id,x_center,y_center,bbox_w,bbox_h)
    return label


In [3]:
import numpy as np
from PIL import Image
from tqdm import tqdm

def img2label_paths(img_paths):
    # Define label paths as a function of image paths
    sa, sb = os.sep + 'images' + os.sep, os.sep + 'labels' + os.sep  # /images/, /labels/ substrings
    return [sb.join(x.rsplit(sa, 1)).rsplit('.', 1)[0] + '.txt' for x in img_paths]


#json_files = ['flag_ann_train_all.json','flag_ann_val_all.json','flag_ann_test_all.json']
json_files = ['flag_ann_finaltest_update.json']
img_root = '/data/flag/test_dataset_label'
json_root = '/data/flag'
save_root = '/data/flag/test_dataset_label/labels'
for js in json_files:
    json_path = os.path.join(json_root,js)
    with open(json_path) as f:
        imgs = json.load(f)
    imgpath_array = []
    imgpath_path = os.path.join(img_root,js[:-4]+'txt')
    print(imgpath_path)
    for img in tqdm(imgs):
        txt_array = []
        img_path = img['image_path']
        if not os.path.exists(img_path):
            img_path = os.path.join(img_root,'images',img['image_path'])
        assert os.path.exists(img_path),'error path'
        imgpath_array.append(img_path+'\n')
        #txt_path = os.path.join(save_root,img['image_path'][:-3]+'txt')
        txt_path = img2label_paths([img_path])[0]
        if os.path.exists(txt_path):
            continue
        if not os.path.exists(os.path.dirname(txt_path)):
            os.makedirs(os.path.dirname(txt_path))
        #print(txt_path)
        img_file = Image.open(img_path)
        w,h = img_file.size
        anno = img['annotation']
        for an in anno:
            class_id = an['class_id']
            bbox = an['bbox']
            label = generate_label(bbox,w,h,class_id)
            txt_array.append(label)
        write_txt(txt_array,txt_path)
    write_txt(imgpath_array,imgpath_path)
    #assert 0

100%|██████████| 1502/1502 [00:00<00:00, 24415.07it/s]

/data/flag/test_dataset_label/flag_ann_finaltest_update.txt


In [27]:
len(imgpath_array)

728